In [3]:
! pip install pymongo scikit-learn python-dotenv

   ---------------------------------------- 0.0/680.4 kB ? eta -:--:--
    --------------------------------------- 10.2/680.4 kB ? eta -:--:--
   -- ------------------------------------ 41.0/680.4 kB 495.5 kB/s eta 0:00:02
   ----- --------------------------------- 92.2/680.4 kB 871.5 kB/s eta 0:00:01
   ---------- ----------------------------- 174.1/680.4 kB 1.1 MB/s eta 0:00:01
   ------------------ --------------------- 307.2/680.4 kB 1.5 MB/s eta 0:00:01
   ----------------------------- ---------- 501.8/680.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 680.4/680.4 kB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/307.7 kB ? eta -:--:--
   --------------------------------------- 307.7/307.7 kB 19.8 MB/s eta 0:00:00


# Import Libraries


In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Connect with MongoDB 

In [ ]:
MONGO_CONNECTION_STRING = ''
clint = MongoClient(MONGO_CONNECTION_STRING)
db = clint['traffic sign']
train_collection = db['train_collection']
validation_collection = db['validation_collection']

data = list(collection.find())
df = pd.DataFrame(data)

df.drop('_id', axis=1, inplace=True)
df.head()

# Task 1

## Load the data and preprocessing

In [20]:
batch_size = 64
img_height = 64
img_width = 64
data = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_image = data.flow_from_directory(
    r'C:\Users\SaadL\OneDrive\المستندات\for_VS\T5_Week4_MProj\traffic_sign_dataset\train',
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode='categorical',
    subset='training'
)

validation_image = data.flow_from_directory(
    r'C:\Users\SaadL\OneDrive\المستندات\for_VS\T5_Week4_MProj\traffic_sign_dataset\train',
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3465 images belonging to 2 classes.
Found 866 images belonging to 2 classes.


## Data Augmentation

In [21]:
Data_generator_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range =0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearset'
)
aug_train_image = Data_generator_train.fit(train_image)

KeyboardInterrupt: 